In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from Kick import kick
import cloudpickle

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.dropout1 = nn.Dropout2d(p=0.25)
        self.dropout2 = nn.Dropout2d(p=0.5)
        self.fc1 = nn.Linear(in_features=64*12*12, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
model = Net()
optimizer = optim.Adadelta(model.parameters(), lr=1e-2)

In [4]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('data', train=False, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])), batch_size=10, shuffle=True)
dataiter = iter(train_loader)

In [5]:
@kick("gpu")
def run():
    for batch_idx, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(X)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print("loss at epoch", batch_idx, loss.item())
    return model

>> initialize to gpu


In [6]:
a = run()